In [34]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests

print('Libraries imported.')


Libraries imported.


In [7]:
#Data from Segmenting and Clustering Neighborhoods in Toronto - week3

import pandas as pd
from pandas.io.html import read_html
import numpy as np
from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wikitables = read_html(url, attrs={'class':'wikitable'})

tables = wikitables[0]
tables.columns = tables.iloc[0]
tables.head()
new_tables = tables.drop((tables.index[0]))

#Clean Data
new_tables.head()
new_tables_drop = new_tables[new_tables.Borough != 'Not assigned']
new_tables_drop = new_tables_drop.reset_index(drop=True)
new_tables_drop.head()

clean_table = new_tables_drop.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
clean_table_new=clean_table.reset_index()
clean_table_new.head(15)

!wget -q -O 'Geospatial_Coordinates.csv'  http://cocl.us/Geospatial_data
df_Geospatial_Coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_Geospatial_Coordinates.head()
df_Geospatial_Coordinates.columns=['Postcode','Latitude','Longitude']

#Merge in Postcode
df_Toronto = pd.merge(clean_table_new,
                 df_Geospatial_Coordinates[['Postcode','Latitude', 'Longitude']],
                 on='Postcode')
df_Toronto.head(9)

'wget' is not recognized as an internal or external command,
operable program or batch file.


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937


In [6]:
set(df_Toronto['Borough'])

{'Central Toronto',
 'Downtown Toronto',
 'East Toronto',
 'East York',
 'Etobicoke',
 'Mississauga',
 'North York',
 "Queen's Park",
 'Scarborough',
 'West Toronto',
 'York'}

In [23]:
df_Toronto[df_Toronto['Borough'] == 'Central Toronto']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197
68,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
79,M4S,Central Toronto,Davisville,43.704324,-79.388790
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
86,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [24]:
df_Toronto[df_Toronto['Borough'] == 'Downtown Toronto']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [25]:
df_Toronto[df_Toronto['Borough'] == 'East Toronto']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
47,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
54,M4M,East Toronto,Studio District,43.659526,-79.340923
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [42]:
df_Toronto[df_Toronto['Borough'] == 'East York'].shape

(5, 5)

In [41]:
df_Toronto[df_Toronto['Borough'] == 'Etobicoke'].shape

(12, 5)

In [40]:
df_Toronto[df_Toronto['Borough'] == 'Mississauga'].shape

(1, 5)

In [39]:
df_Toronto[df_Toronto['Borough'] == 'North York'].shape

(24, 5)

In [37]:
df_Toronto[df_Toronto['Borough'] == "Queen's Park"].shape

(1, 5)

In [35]:
df_Toronto[df_Toronto['Borough'] == 'Scarborough'].shape

(17, 5)

In [36]:
df_Toronto[df_Toronto['Borough'] == 'West Toronto'].shape

(6, 5)

In [38]:
df_Toronto[df_Toronto['Borough'] == 'York'].shape

(5, 5)

We consider the number of postcodes and the amount of the neighborhood in which 'North York' exceeding other postcodes.

In [43]:
df_Toronto[df_Toronto['Borough'] == 'North York']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
7,M3B,North York,Don Mills North,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073
13,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
